Ноутбук запускался в колабе

In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device == torch.device('cpu'):
    print('Using cpu')
else:
    n_gpu = torch.cuda.device_count()
    print('Using {} GPUs'.format(torch.cuda.get_device_name(0)))

Using Tesla T4 GPUs


In [2]:
! pip install datasets transformers seqeval

     |████████████████████████████████| 225kB 6.4MB/s 
     |████████████████████████████████| 2.3MB 22.1MB/s 
     |████████████████████████████████| 51kB 9.0MB/s 
     |████████████████████████████████| 245kB 46.7MB/s 
     |████████████████████████████████| 112kB 52.1MB/s 
     |████████████████████████████████| 901kB 50.5MB/s 
     |████████████████████████████████| 3.3MB 49.3MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=8986272abeafd847f6dbd1cf76a7955d767f7191629f01c39481b44d97637e96
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


In [3]:
task = "ner"
batch_size = 8

# Data - PAD

In [4]:
!gdown --id 1cCXjHX9FAgouF0DWuWPO5qyBHVZDkXTQ

Downloading...
From: https://drive.google.com/uc?id=1cCXjHX9FAgouF0DWuWPO5qyBHVZDkXTQ
To: /content/NER_PAD_agg_test.csv
2.92MB [00:00, 91.5MB/s]


In [5]:
!gdown --id 1RFNBRcly96omdpNtYlEK6O1EuQs3ux7t

Downloading...
From: https://drive.google.com/uc?id=1RFNBRcly96omdpNtYlEK6O1EuQs3ux7t
To: /content/NER_PAD_agg_train.csv
11.7MB [00:00, 36.9MB/s]


In [6]:
!gdown --id 1AnLqXtSyJNBK7YmwuW6L6z2OYhE4RPTV

Downloading...
From: https://drive.google.com/uc?id=1AnLqXtSyJNBK7YmwuW6L6z2OYhE4RPTV
To: /content/NER_PAD_agg.csv
14.6MB [00:00, 68.2MB/s]


In [7]:
import pandas as pd
data = pd.read_csv('NER_PAD_agg.csv', converters={'tokens': eval, 'ner_tags': eval})

In [8]:
all_tags = set()
for _, row in data.iterrows():
    for tag in row.ner_tags:
        all_tags.add(tag)
all_tags = sorted(list(all_tags))

In [9]:
tag_to_idx = {t: i for i, t in enumerate(all_tags)}

In [10]:
label_list = {i: t for i, t in enumerate(all_tags)}

In [11]:
train = pd.read_csv('NER_PAD_agg_train.csv', converters={'tokens': eval, 'ner_tags': eval})
test = pd.read_csv('NER_PAD_agg_test.csv', converters={'tokens': eval, 'ner_tags': eval})

In [12]:
train.ner_tags = train.ner_tags.apply(lambda x : [tag_to_idx[label] for label in x])
test.ner_tags = test.ner_tags.apply(lambda x : [tag_to_idx[label] for label in x])

In [13]:
train.to_pickle('train.pkl')
test.to_pickle('test.pkl')

In [14]:
from datasets import load_dataset, load_metric

In [15]:
datasets = load_dataset('pandas', data_files={'train' : 'train.pkl', 'test' : 'test.pkl'})

Using custom data configuration default-1d0f8a4de0b19a70


Dataset pandas downloaded and prepared to /root/.cache/huggingface/datasets/pandas/default-1d0f8a4de0b19a70/0.0.0/3547ff13f14ebf30dc8f11ab416dcf7721c18de2eb7e054bf9c80b88e765a791. Subsequent calls will reuse this data.


In [16]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3200
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 800
    })
})

# Tokenizer

In [17]:
from transformers import AutoTokenizer
model_checkpoint = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [18]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [19]:
label_all_tokens = False

In [20]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [21]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Token indices sequence length is longer than the specified maximum sequence length for this model (954 > 512). Running this sequence through the model will result in indexing errors


Warning : есть последовательности больше, чем максимально возможная в BERT - 512

In [22]:
pd.Series(tokenized_datasets["train"]['input_ids']).apply(lambda x : len(x)).describe()

count    3200.000000
mean      454.245000
std       283.854426
min         6.000000
25%       207.000000
50%       405.000000
75%       566.000000
max      4271.000000
dtype: float64

In [23]:
pd.Series(tokenized_datasets["test"]['input_ids']).apply(lambda x : len(x)).describe()

count     800.000000
mean      453.612500
std       281.724152
min        47.000000
25%       202.000000
50%       406.000000
75%       566.500000
max      1796.000000
dtype: float64

In [24]:
train_tokenized = tokenized_datasets["train"].to_pandas()
test_tokenized = tokenized_datasets["test"].to_pandas()

In [25]:
train_tokenized.head(1)

,attention_mask,id,input_ids,labels,ner_tags,token_type_ids,tokens
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",SV990115834_ОРГ__INM_16'04'09_3.pdf,"[101, 180, 37410, 10519, 125, 118, 190, 94389,...","[-100, 15, 15, -100, 15, -100, 15, 15, -100, -...","[15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[l, испр, 4-, v, moscow, quarterly, report, li..."


In [26]:
test_tokenized.head(1)

,attention_mask,id,input_ids,labels,ner_tags,token_type_ids,tokens
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",B31005082_ОРГ__INM_15'39'47_1.pdf,"[101, 180, 559, 106, 62213, 1779, 122, 80765, ...","[-100, 15, 15, 15, 15, 15, 15, 15, -100, -100,...","[15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[l, т, !, приложение, №, 1, постановлению, пра..."


In [27]:
CHUNCK_SIZE = 512

def lists_to_chuncks(row):
    n = CHUNCK_SIZE - 2
    new_row = row.copy()
    for key in ['attention_mask', 'input_ids', 'labels', 'token_type_ids']:
        FIRST = new_row[key][0]
        LAST = new_row[key][-1]
        new_row[key] = new_row[key][1:-1]
        new_row[key] = [[FIRST] + list(new_row[key][i:i + n]) + [LAST] for i in range(0, len(new_row[key]), n)]
    return new_row

In [28]:
train_tokenized = train_tokenized.drop(columns = ['ner_tags', 'tokens']).apply(lists_to_chuncks, axis=1)
train_tokenized = train_tokenized.set_index(['id']).apply(pd.Series.explode).reset_index()

In [29]:
test_tokenized = test_tokenized.drop(columns = ['ner_tags', 'tokens']).apply(lists_to_chuncks, axis=1)
test_tokenized = test_tokenized.set_index(['id']).apply(pd.Series.explode).reset_index()

In [30]:
train_tokenized.to_pickle('train_tokenized.pkl')
test_tokenized.to_pickle('test_tokenized.pkl')

In [31]:
tokenized_datasets = load_dataset('pandas', data_files={'train' : 'train_tokenized.pkl', 'test' : 'test_tokenized.pkl'})

Using custom data configuration default-4583f61dcbe3f83e


Dataset pandas downloaded and prepared to /root/.cache/huggingface/datasets/pandas/default-4583f61dcbe3f83e/0.0.0/3547ff13f14ebf30dc8f11ab416dcf7721c18de2eb7e054bf9c80b88e765a791. Subsequent calls will reuse this data.


In [32]:
pd.Series(tokenized_datasets["train"]['input_ids']).apply(lambda x : len(x)).describe()

count    4415.000000
mean      329.787995
std       162.203175
min         3.000000
25%       184.000000
50%       358.000000
75%       512.000000
max       512.000000
dtype: float64

In [33]:
pd.Series(tokenized_datasets["test"]['input_ids']).apply(lambda x : len(x)).describe()

count    1090.000000
mean      333.458716
std       159.904559
min         3.000000
25%       183.000000
50%       369.500000
75%       512.000000
max       512.000000
dtype: float64

Теперь все последовательности нужной длины

# Model

In [34]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

Warning : модель надо обучить на наших данных

In [35]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [36]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [37]:
metric = load_metric("seqeval")

In [38]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [39]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [40]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.124300,0.050599,0.768081,0.801744,0.784551,0.985188
2,0.046000,0.040963,0.802826,0.860486,0.830657,0.987651
3,0.034500,0.039726,0.825158,0.868518,0.846283,0.988453


TrainOutput(global_step=1656, training_loss=0.0645401486452075, metrics={'train_runtime': 1667.2688, 'train_samples_per_second': 0.993, 'total_flos': 675171562108416.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 2026078208, 'init_mem_gpu_alloc_delta': 709382656, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -247537664, 'train_mem_gpu_alloc_delta': 2197179392, 'train_mem_cpu_peaked_delta': 655101952, 'train_mem_gpu_peaked_delta': 6504072192})

In [41]:
trainer.evaluate()

{'epoch': 3.0,
 'eval_accuracy': 0.9884531831578365,
 'eval_f1': 0.8462828395751818,
 'eval_loss': 0.039726145565509796,
 'eval_mem_cpu_alloc_delta': 2039808,
 'eval_mem_cpu_peaked_delta': 1777664,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 415944704,
 'eval_precision': 0.8251580553738828,
 'eval_recall': 0.8685176686553465,
 'eval_runtime': 44.1667,
 'eval_samples_per_second': 24.679}

In [42]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'DOCAGRDATE': {'f1': 0.7422096317280454,
  'number': 151,
  'precision': 0.6485148514851485,
  'recall': 0.8675496688741722},
 'DOCAGRNUM': {'f1': 0.5771428571428572,
  'number': 179,
  'precision': 0.5906432748538012,
  'recall': 0.5642458100558659},
 'DOCAMOUNT': {'f1': 0.8735543562066306,
  'number': 1244,
  'precision': 0.8392592592592593,
  'recall': 0.9107717041800643},
 'DOCCPTY': {'f1': 0.82398753894081,
  'number': 596,
  'precision': 0.7688953488372093,
  'recall': 0.8875838926174496},
 'DOCCPTYINN': {'f1': 0.9482758620689654,
  'number': 282,
  'precision': 0.9228187919463087,
  'recall': 0.975177304964539},
 'DOCCUSTOMER': {'f1': 0.9175019275250578,
  'number': 610,
  'precision': 0.86608442503639,
  'recall': 0.9754098360655737},
 'DOCDATE': {'f1': 0.8987915407854986,
  'number': 645,
  'precision': 0.8762886597938144,
  'recall': 0.9224806201550387},
 'DOCNUM': {'f1': 0.7325881341358557,
  'number': 651,
  'precision': 0.83203125,
  'recall': 0.6543778801843319},
 'overa

In [43]:
pd.DataFrame.from_dict(results).transpose().iloc[:-4, :-1]

,precision,recall,f1
DOCAGRDATE,0.648515,0.867550,0.742210
DOCAGRNUM,0.590643,0.564246,0.577143
DOCAMOUNT,0.839259,0.910772,0.873554
DOCCPTY,0.768895,0.887584,0.823988
DOCCPTYINN,0.922819,0.975177,0.948276
DOCCUSTOMER,0.866084,0.975410,0.917502
DOCDATE,0.876289,0.922481,0.898792
DOCNUM,0.832031,0.654378,0.732588


In [44]:
pd.DataFrame.from_dict(results).transpose().iloc[-4:, 0].rename('result')

overall_precision    0.825158
overall_recall       0.868518
overall_f1           0.846283
overall_accuracy     0.988453
Name: result, dtype: float64

In [45]:
!pip install sklearn_crfsuite

     |████████████████████████████████| 747kB 10.9MB/s 


In [46]:
from sklearn_crfsuite import metrics
print(metrics.flat_classification_report(true_labels, true_predictions, labels=all_tags[:-1], digits=3))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


               precision    recall  f1-score   support

 B-DOCAGRDATE      0.691     0.893     0.779       150
  B-DOCAGRNUM      0.632     0.603     0.617       179
  B-DOCAMOUNT      0.862     0.920     0.890      1243
    B-DOCCPTY      0.828     0.909     0.867       595
 B-DOCCPTYINN      0.923     0.975     0.948       282
B-DOCCUSTOMER      0.900     0.974     0.935       609
    B-DOCDATE      0.910     0.936     0.923       645
     B-DOCNUM      0.832     0.654     0.733       651
 I-DOCAGRDATE      0.633     0.784     0.701        88
  I-DOCAGRNUM      0.000     0.000     0.000         7
  I-DOCAMOUNT      0.814     0.894     0.852       620
    I-DOCCPTY      0.812     0.917     0.861       564
I-DOCCUSTOMER      0.859     0.965     0.909       462
    I-DOCDATE      0.921     0.940     0.930       605
     I-DOCNUM      0.000     0.000     0.000         0

    micro avg      0.849     0.893     0.870      6700
    macro avg      0.708     0.758     0.730      6700
 weighte